In [2]:
import os
from typing import AnyStr, Union

from arcgis.gis import GIS, User
from arcgis import geoenrichment as ge
from arcgis.geoenrichment import _ge
import arcpy
from dm import utils, Country
from dotenv import find_dotenv, load_dotenv
import pandas as pd

load_dotenv(find_dotenv())

True

In [1]:
def get_countries_local():
    
    # get a generator of dataset objects 
    ds_lst = list(arcpy._ba.ListDatasets())

    # throw error if no local datasets are available
    assert len(ds_lst), 'No datasets are available locally. If you want to locate available countries on a Web GIS, please provide a GIS object instance as for the source parameter.'

    # organize all the country dataset properites
    cntry_lst = [(ds.CountryInfo.Name, ds.Version, ds.CountryInfo.ISO2, ds.CountryInfo.ISO3, ds.Caption, ds.DataSourceID, ds.ID) for ds in ds_lst]

    # create a dataframe of all the country properties
    cntry_df = pd.DataFrame(cntry_lst, columns=['country_name', 'vintage', 'iso2', 'iso3', 'description', 'data_source_id', 'country_id'])
    
    return cntry_df
    

def get_countries_gis(source):
    
    # make sure countries are available
    ge_err_msg = 'The GIS instance you proided does not appear to have geoenrichment enabled and configured, so no countries are available.'
    assert 'geoenrichment' in source.properties.helperServices, ge_err_msg
    assert isinstance(source.properties.helperServices.geoenrichment['url'], str), ge_err_msg

    # extract out the geoenrichment url
    ge_url = source.properties.helperServices.geoenrichment['url']

    # get a list of countries available on the Web GIS for enrichment
    url = f'{ge_url}/Geoenrichment/Countries'
    cntry_res = source._con.post(url, {'f' : 'json'})
    cntry_dict = cntry_res['countries']

    # convert the dictionary to a dataframe
    cntry_df = pd.DataFrame(cntry_dict)
    
    return cntry_df


def get_countries(source:Union[GIS, AnyStr]='local')->pd.DataFrame:
    """
    Get the available countries based on the enrichment source.
    """
    
    # ensure the source is as expected
    assert isinstance(source, (str, GIS)), f'Please ensure the "source" is either the "local" keyword, or a valid GIS object instance instead of "{type(source)}".'
    
    # if the source is local
    if isinstance(source, str):
        
        # make sure the correct keyword is being used to avoid confusion
        assert source.lower() == 'local', f'If you are interested in countries available locally, please use the "local" keyword for the "source" parameter. You provided "{source}".'
        
        # get the local country dataframe
        cntry_df = get_country_local()
    
    # if we are using accessing a Web GIS object instance
    elif isinstance(source, GIS):
        
        # get gis enrichment countries available
        cntry_df = get_country_gis(sourc)
    
    return cntry_df


NameError: name 'Union' is not defined

In [32]:
gis = GIS(os.getenv('ESRI_GIS_URL'), username=os.getenv('ESRI_GIS_USERNAME'), password=os.getenv('ESRI_GIS_PASSWORD'))

gis

GIS @ https://geoai.maps.arcgis.com version:8.4

In [37]:
user = gis.users.me
prv_tail_lst = [prv.split(':')[-1] for prv in user['privileges'] if 'networkanalysis' in prv]
print(prv_tail_lst)

['networkanalysis', 'closestfacility', 'locationallocation', 'optimizedrouting', 'origindestinationcostmatrix', 'routing', 'servicearea', 'vehiclerouting']


In [126]:
cntry_df = get_countries(gis)

cntry_df.head()

,id,name,abbr3,altName,continent,distanceUnits,esriUnits,defaultExtent,defaultDatasetID,datasets,hierarchies,defaultDataCollection,dataCollections,defaultReportTemplate,currencySymbol,currencyFormat
0,AL,Albania,ALB,ALBANIA,Europe,Kilometers,esriKilometers,"{'xmin': 19.2627565462, 'ymin': 39.6444975717,...",ALB_MBR_2019,[ALB_MBR_2019],"[{'ID': 'census', 'alias': 'AL Standard Data',...",KeyGlobalFacts,,Albania Summary Report,ALL,ALL 0;-ALL 0
1,DZ,Algeria,DZA,ALGERIA,Africa,Kilometers,esriKilometers,"{'xmin': -8.6738679419, 'ymin': 18.9600292058,...",DZA_MBR_2019,[DZA_MBR_2019],"[{'ID': 'census', 'alias': 'DZ Standard Data',...",KeyGlobalFacts,,Algeria Summary Report,DZD,DZD 0;-DZD 0
2,AD,Andorra,AND,ANDORRA,Europe,Kilometers,esriKilometers,"{'xmin': 1.4087179566, 'ymin': 42.4287636263, ...",AND_MBR_2019,[AND_MBR_2019],"[{'ID': 'census', 'alias': 'AD Standard Data',...",KeyGlobalFacts,,Andorra Summary Report,€,€0;-€0
3,AO,Angola,AGO,ANGOLA,Africa,Kilometers,esriKilometers,"{'xmin': 11.6792204648, 'ymin': -18.0420776812...",AGO_MBR_2019,[AGO_MBR_2019],"[{'ID': 'census', 'alias': 'AO Standard Data',...",KeyGlobalFacts,,Angola Summary Report,AOA,AOA 0;-AOA 0
4,AR,Argentina,ARG,ARGENTINA,South America,Kilometers,esriKilometers,"{'xmin': -73.5777799704, 'ymin': -55.061314686...",ARG_MBR_2018,[ARG_MBR_2018],"[{'ID': 'census', 'alias': 'AR Standard Data',...",KeyGlobalFacts,,Argentina Summary Report,ARS,ARS 0;-ARS 0


In [177]:
nxt_df = cntry_df[['name', 'id', 'abbr3']].copy()
nxt_df.columns = ['country_name', 'iso2', 'iso3']

nxt_df.head()

,country_name,iso2,iso3
0,Albania,AL,ALB
1,Algeria,DZ,DZA
2,Andorra,AD,AND
3,Angola,AO,AGO
4,Argentina,AR,ARG


,country_name,iso2,iso3,country_id,vintage


In [130]:
lcl_df = get_countries()

lcl_df.head()

,country_name,vintage,iso2,iso3,description,data_source_id,country_id
0,Canada,2019,CA,CAN,Esri Canada 2019 Data,LOCAL;;CAN_ESRI_2019,CAN_ESRI_2019
1,United States,2019,US,USA,Esri USA 2019,LOCAL;;USA_ESRI_2019,USA_ESRI_2019
2,United States,2020,US,USA,Esri USA 2020,LOCAL;;USA_ESRI_2020,USA_ESRI_2020


In [9]:
ds_list = list(arcpy._ba.ListDatasets())
ds = ds_list[0]

In [11]:
ds.__dict__

{'ID': 'CAN_ESRI_2019',
 'Caption': 'Esri Canada 2019 Data',
 'CountryInfo': <arcpy._ba.CountryInfo at 0x21e63856788>,
 'DataSourceID': 'LOCAL;;CAN_ESRI_2019',
 'Version': '2019'}

In [17]:
var_lst = list(arcpy._ba.ListVariables('CAN_ESRI_2019'))
var = var_lst[0]

In [19]:
var.__dict__

{'Name': 'A16AITFNAT',
 'FullName': 'AboriginalIdentity.A16AITFNAT',
 'Alias': '2016 First Nations Single Ident',
 'DataCollectionID': 'AboriginalIdentity',
 'OutputFieldName': 'AboriginalIdentity_A16AITFNAT'}

In [23]:
from dm import Country, country

In [21]:
cntry = Country('CAN')

In [30]:
dm_df = country.get_enrich_variables_dataframe('CAN')
dm_df.head()

,name,alias,type,vintage,data_collection,enrich_str,enrich_field_name
0,A16AITFNAT,2016 First Nations Single Ident,COUNT,2016,AboriginalIdentity,AboriginalIdentity.A16AITFNAT,AboriginalIdentity_A16AITFNAT
1,A16AITIDT,2016 Aboriginal Identity,COUNT,2016,AboriginalIdentity,AboriginalIdentity.A16AITIDT,AboriginalIdentity_A16AITIDT
2,A16AITIDX,2016 Aboriginal Identities,COUNT,2016,AboriginalIdentity,AboriginalIdentity.A16AITIDX,AboriginalIdentity_A16AITIDX
3,A16AITINUK,2016 Inuk Single Identity,COUNT,2016,AboriginalIdentity,AboriginalIdentity.A16AITINUK,AboriginalIdentity_A16AITINUK
4,A16AITMETI,2016 Metis Single Identity,COUNT,2016,AboriginalIdentity,AboriginalIdentity.A16AITMETI,AboriginalIdentity_A16AITMETI


In [31]:
e_df = pd.DataFrame( ((v.Name, v.Alias, v.DataCollectionID, v.FullName, v.OutputFieldName) for v in arcpy._ba.ListVariables('CAN_ESRI_2019')), 
                    columns=['name', 'alias', 'data_collection', 'enrich_name', 'enrich_field_name'])

e_df.head()

,name,alias,data_collection,enrich_name,enrich_field_name
0,A16AITFNAT,2016 First Nations Single Ident,AboriginalIdentity,AboriginalIdentity.A16AITFNAT,AboriginalIdentity_A16AITFNAT
1,A16AITIDT,2016 Aboriginal Identity,AboriginalIdentity,AboriginalIdentity.A16AITIDT,AboriginalIdentity_A16AITIDT
2,A16AITIDX,2016 Aboriginal Identities,AboriginalIdentity,AboriginalIdentity.A16AITIDX,AboriginalIdentity_A16AITIDX
3,A16AITINUK,2016 Inuk Single Identity,AboriginalIdentity,AboriginalIdentity.A16AITINUK,AboriginalIdentity_A16AITINUK
4,A16AITMETI,2016 Metis Single Identity,AboriginalIdentity,AboriginalIdentity.A16AITMETI,AboriginalIdentity_A16AITMETI
